# Example of Stackstack Issue

This notebook illustrates the inconsistency between the xarrays produced by stackstac vs rioxarray.

In [24]:
%load_ext autoreload
%autoreload 2
import nisardev as nisar
import os
import gimpfunc as gimp
import panel
import numpy as np
panel.extension()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
env = dict(GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/.gimp_download_cookiejar.txt'),
            GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/.gimp_download_cookiejar.txt'))
os.environ.update(env)

In [26]:
myLogin = gimp.NASALogin()
myLogin.view()

Row
    [0] ParamMethod(method)
    [1] Column(margin=5, name='Earth Data Login', width=300)
        [0] StaticText(value='<b>Earth Data Login</b>')
        [1] TextInput(name='Username')
        [2] PasswordInput(name='Password')
        [3] Button(name='Enter Credentials')
    [2] Column
        [0] ParamMethod(method)
        [1] ParamMethod(method)

In [27]:
httpName = 'https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0725.003/2019.12.01/GL_vel_mosaic_Annual_01Dec19_30Nov20_*_v03.0'

In [28]:
products = gimp.GIMPSubsetter(urls=[httpName],  bands=['vv', 'vx']) # vx,vy,vy, ex, ey, dT
products.loadStackStac()
products.DA

,Array,Chunk
Bytes,792.97 MiB,1.00 MiB
Shape,"(1, 2, 13701, 7586)","(1, 1, 512, 512)"
Count,1219 Tasks,810 Chunks
Type,float32,numpy.ndarray


In [29]:
myVelRemote = nisar.nisarVel()
myVelRemote.readDataFromTiff(httpName, url=True, readSpeed=False)
myVelRemote.xr

,Array,Chunk
Bytes,1.16 GiB,1.00 MiB
Shape,"(1, 3, 13700, 7585)","(1, 1, 512, 512)"
Count,7292 Tasks,1215 Chunks
Type,float32,numpy.ndarray


In [30]:
print('rioxr xmin',np.min(myVelRemote.xr.x.data), 'stack xmin',np.min(products.DA.x.data))
print('rioxr xmax',np.max(myVelRemote.xr.x.data), 'stack xmax',np.max(products.DA.x.data))
print('rioxr ymin',np.min(myVelRemote.xr.y.data), 'stack ymin',np.min(products.DA.y.data))
print('rioxr ymax',np.max(myVelRemote.xr.y.data), 'stack ymax',np.max(products.DA.y.data))
print('rioxr shape', myVelRemote.xr.shape, 'stack shape', products.DA.shape)

rioxr xmin -659000.0 stack xmin -659200.0
rioxr xmax 857800.0 stack xmax 857800.0
rioxr ymin -3379000.0 stack ymin -3379000.0
rioxr ymax -639200.0 stack ymax -639000.0
rioxr shape (1, 3, 13700, 7585) stack shape (1, 2, 13701, 7586)


Note xmax agree for both, but are off by 200 m for xmin. For y, the mins agree, but the maxes are off. The stack array is 1 pixel bigger in each dimension. I couldn't find any simple tweak of the input bounds that would fix this. 